# Exploring and visualizing DICOM SR
The purpose of this notebook is to provide guidance when attempting to explore and visualize certain DICOM data sets as provided by [AIDA](https://datahub.aida.scilifelab.se/). In particular, datasets containing DICOM SR. For example, the following datasets:
* [CTPA](https://datahub.aida.scilifelab.se/)
* [CTPEL](https://datahub.aida.scilifelab.se/10.23698/aida/ctpel)
* [DRLI](https://datahub.aida.scilifelab.se/10.23698/aida/drli)
* [DRSKE](https://datahub.aida.scilifelab.se/10.23698/aida/drske)

In the following cells, it is assumed that one examination from any of these examinations are available locally.

## Preparations

In [1]:
# Imports
%matplotlib inline
from pathlib import Path
import pydicom

Specify data to explore and visualize by providing a path to a folder. It is assumed that the specified folder contains a single examination and that all relevant files end with `.dcm`.

In [2]:
folder = Path("/mnt/c/Users/da-for/Downloads/0FC3188AAA7E6851")

Loop over all files to understand the content and to create a mapping structure.


In [3]:
sr_files = []

for dcm_file in folder.rglob("*.dcm"):
    ds = pydicom.read_file(dcm_file)
    if ds.Modality == "SR":
        sr_files.append(dcm_file)

## DICOM SR
A DICOM SR object (SR = Structured Report) is a DICOM object that comes in many flavours. For example:
* Basic Text SR
* Enhanced SR
* Comprehensive SR
* Key Object Selection Document
* Mammography CAD SR
* Chest CAD SR
Even with a single of these IODs, there can exist a variety of templates that can be used to create these object, which in turn often allows for a lot of complexity to be added.

In this case, we will focus on Enhanced SR or Comprehensive SR, using [TID 1500](http://dicom.nema.org/medical/dicom/current/output/chtml/part16/chapter_A.html#sect_TID_1500), i.e., Measurement Report. This is the DICOM SR object and template suggested in [IHE Radiology Technical Framework Supplement - AI Results (AIR)](https://www.ihe.net/uploadedFiles/Documents/Radiology/IHE_RAD_Suppl_AIR.pdf) to use when communicating findings/observations from AI applications. These SR objects and this templte can of course also be used for human generated findings/observation.

In the following, we assume that at least one SEG file was found and that we use the first one.

In [4]:
sr_file = sr_files[0]
sr_ds = pydicom.read_file(sr_file)
print(f"The SR file {sr_file} is selected for exploration")

The SR file /mnt/c/Users/da-for/Downloads/0FC3188AAA7E6851/im_6/x0000.dcm is selected for exploration


In the following cells, the content of the SR file will be explored step by step.

The `ConceptNameCodeSequence` describes the type of report

In [5]:
print(sr_ds.ConceptNameCodeSequence[0])

(0008, 0100) Code Value                          SH: '126000'
(0008, 0102) Coding Scheme Designator            SH: 'DCM'
(0008, 0104) Code Meaning                        LO: 'Imaging Measurement Report'


The `ContentTemplateSequence` describes the template used to create the SR report

In [6]:
print(sr_ds.ContentTemplateSequence[0])

(0008, 0105) Mapping Resource                    CS: 'DCMR'
(0008, 0118) Mapping Resource UID                UI: DICOM Content Mapping Resource
(0040, db00) Template Identifier                 CS: '1500'


Most of the relevant content is available in the `ContentSequence`

In [7]:
print(f"The top level content of the ContentSequence is:")
im_ind = None
ind = 0
for item in sr_ds.ContentSequence:
    print(item.ConceptNameCodeSequence[0])
    print(f"")
    if item.ConceptNameCodeSequence[0].CodeValue == "126010":
        im_ind = ind
    ind += 1
im_item = sr_ds.ContentSequence[im_ind]

The top level content of the ContentSequence is:
(0008, 0100) Code Value                          SH: '121049'
(0008, 0102) Coding Scheme Designator            SH: 'DCM'
(0008, 0104) Code Meaning                        LO: 'Language of Content Item and Descendants'

(0008, 0100) Code Value                          SH: '121058'
(0008, 0102) Coding Scheme Designator            SH: 'DCM'
(0008, 0104) Code Meaning                        LO: 'Procedure Reported'

(0008, 0100) Code Value                          SH: '111028'
(0008, 0102) Coding Scheme Designator            SH: 'DCM'
(0008, 0104) Code Meaning                        LO: 'Image Library'

(0008, 0100) Code Value                          SH: '126010'
(0008, 0102) Coding Scheme Designator            SH: 'DCM'
(0008, 0104) Code Meaning                        LO: 'Imaging Measurements'



The last item will contain the Imaging Measurements.

In [8]:
print(f"The imaging measurement item has a content of {len(im_item.ContentSequence)} measurement groups:")
print(im_item.ContentSequence[0].ConceptNameCodeSequence[0])

The imaging measurement item has a content of 15 measurement groups:
(0008, 0100) Code Value                          SH: '125007'
(0008, 0102) Coding Scheme Designator            SH: 'DCM'
(0008, 0104) Code Meaning                        LO: 'Measurement Group'


Each measurement group has the following content:

In [9]:
for mg in im_item.ContentSequence[0].ContentSequence:
    print(mg.ConceptNameCodeSequence[0])
    print(f"")

(0008, 0100) Code Value                          SH: '112039'
(0008, 0102) Coding Scheme Designator            SH: 'DCM'
(0008, 0104) Code Meaning                        LO: 'Tracking Identifier'

(0008, 0100) Code Value                          SH: '112040'
(0008, 0102) Coding Scheme Designator            SH: 'DCM'
(0008, 0104) Code Meaning                        LO: 'Tracking Unique Identifier'

(0008, 0100) Code Value                          SH: '121071'
(0008, 0102) Coding Scheme Designator            SH: 'DCM'
(0008, 0104) Code Meaning                        LO: 'Finding'

(0008, 0100) Code Value                          SH: '363698007'
(0008, 0102) Coding Scheme Designator            SH: 'SCT'
(0008, 0104) Code Meaning                        LO: 'Finding Site'

(0008, 0100) Code Value                          SH: '758637006'
(0008, 0102) Coding Scheme Designator            SH: 'SCT'
(0008, 0104) Code Meaning                        LO: 'Anatomical locations'



In this case, it is the anatomical locations that contains the so called measurement:

The measurement further defines that this is a center point.

In [10]:
print(im_item.ContentSequence[0].ContentSequence[-1].ConceptCodeSequence[0])

(0008, 0100) Code Value                          SH: '26216008'
(0008, 0102) Coding Scheme Designator            SH: 'SCT'
(0008, 0104) Code Meaning                        LO: 'Center'


Which in turn is inferred from a specific coordinate and SOP Instance.

In [11]:
print(im_item.ContentSequence[0].ContentSequence[-1].ContentSequence[0])

(0040, a010) Relationship Type                   CS: 'INFERRED FROM'
(0040, a040) Value Type                          CS: 'SCOORD'
(0040, a730)  Content Sequence  1 item(s) ---- 
   (0008, 1199)  Referenced SOP Sequence  1 item(s) ---- 
      (0008, 1150) Referenced SOP Class UID            UI: CT Image Storage
      (0008, 1155) Referenced SOP Instance UID         UI: 1.3.6.1.4.1.9328.50.4.13311
      ---------
   (0040, a010) Relationship Type                   CS: 'SELECTED FROM'
   (0040, a040) Value Type                          CS: 'IMAGE'
   ---------
(0070, 0022) Graphic Data                        FL: [91.0, 283.0]
(0070, 0023) Graphic Type                        CS: 'POINT'
